**Task:-** 

EF-2: Try to make generative model of jet images, using a AUC of a discriminator to distinguish Generative model data / real data as metric

From https://github.com/makagan/SSI_Projects/blob/main/jet_notebooks/1.LHCJetDatasetExploration.ipynbfrom 

In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
 # Data already downloaded 
# let's open the file
fileIN = '../jet_notebooks/Data-MLtutorial/JetDataset/jetImage_7_100p_30000_40000.h5'
f = h5py.File(fileIN)
# and see what it contains
print(list(f.keys()))

['jetConstituentList', 'jetFeatureNames', 'jetImage', 'jetImageECAL', 'jetImageHCAL', 'jets', 'particleFeatureNames']


In [3]:
# These are the quantities we are dealing with
featurenames = f.get('jetFeatureNames')
print(featurenames[:])
# the b is due to the byte vs utf-8 encoding of the strings in the dataset
# just ignore them for the moment

[b'j_ptfrac' b'j_pt' b'j_eta' b'j_mass' b'j_tau1_b1' b'j_tau2_b1'
 b'j_tau3_b1' b'j_tau1_b2' b'j_tau2_b2' b'j_tau3_b2' b'j_tau32_b1'
 b'j_tau32_b2' b'j_zlogz' b'j_c1_b0' b'j_c1_b1' b'j_c1_b2' b'j_c2_b1'
 b'j_c2_b2' b'j_d2_b1' b'j_d2_b2' b'j_d2_a1_b1' b'j_d2_a1_b2' b'j_m2_b1'
 b'j_m2_b2' b'j_n2_b1' b'j_n2_b2' b'j_tau1_b1_mmdt' b'j_tau2_b1_mmdt'
 b'j_tau3_b1_mmdt' b'j_tau1_b2_mmdt' b'j_tau2_b2_mmdt' b'j_tau3_b2_mmdt'
 b'j_tau32_b1_mmdt' b'j_tau32_b2_mmdt' b'j_c1_b0_mmdt' b'j_c1_b1_mmdt'
 b'j_c1_b2_mmdt' b'j_c2_b1_mmdt' b'j_c2_b2_mmdt' b'j_d2_b1_mmdt'
 b'j_d2_b2_mmdt' b'j_d2_a1_b1_mmdt' b'j_d2_a1_b2_mmdt' b'j_m2_b1_mmdt'
 b'j_m2_b2_mmdt' b'j_n2_b1_mmdt' b'j_n2_b2_mmdt' b'j_mass_trim'
 b'j_mass_mmdt' b'j_mass_prun' b'j_mass_sdb2' b'j_mass_sdm1'
 b'j_multiplicity' b'j_g' b'j_q' b'j_w' b'j_z' b'j_t' b'j_undef']


In [4]:
print(f.get('jetImage'))

<HDF5 dataset "jetImage": shape (10000, 100, 100), type "<f8">


In [5]:
jet_data = np.array(f.get('jets'))
target = jet_data[:,-6:-1]
# shape of the dataset
print("Dataset shape:")
print(target.shape)
print("First five entries:")
for i in range(5):
    print(target[i])
print("Last 5 entries:")
for i in range(-5,0):
    print(target[i])

Dataset shape:
(10000, 5)
First five entries:
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 1. 0.]
Last 5 entries:
[0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[1. 0. 0. 0. 0.]


In [6]:
data = np.array(jet_data[:,:-6])
print(data.shape)

(10000, 53)


In [7]:
labelCat= ["gluon", "quark", "W", "Z", "top"]
# this function makes the histogram of a given quantity for the five classes
def makePlot(feature_index, input_data, input_featurenames):
    plt.subplots()
    for i in range(len(labelCat)):
        # notice the use of numpy masking to select specific classes of jets
        my_data = input_data[np.argmax(target, axis=1) == i]
        # then plot the right quantity for the reduced array
        plt.hist(my_data[:,feature_index], 50, density=True, histtype='step', fill=False, linewidth=1.5)
    plt.yscale('log')    
    plt.legend(labelCat, fontsize=12, frameon=False)
    plt.xlabel(str(input_featurenames[feature_index], "utf-8"), fontsize=15)
    plt.ylabel('Prob. Density (a.u.)', fontsize=15)
    plt.show()
    #del fig, ax
    #return fig, ax

# The particle-list dataset
In this case, we look at the particle-related features that we have stored for each jet constituent. The structure of the dataset is similar to that of the physics-motivated features, except for the fact that we have now a double-index dataset: (jet index, particle index). The list is cut at 100 constituents /jet. If less are found, the dataset is completed filling it with 0s (zero padding)

In [8]:
p_featurenames = f.get("particleFeatureNames")
print(p_featurenames[:])

[b'j1_px' b'j1_py' b'j1_pz' b'j1_e' b'j1_erel' b'j1_pt' b'j1_ptrel'
 b'j1_eta' b'j1_etarel' b'j1_etarot' b'j1_phi' b'j1_phirel' b'j1_phirot'
 b'j1_deltaR' b'j1_costheta' b'j1_costhetarel' b'j1_pdgid']


In [9]:
p_data = f.get("jetConstituentList")
print(p_data.shape)

(10000, 100, 16)


In [10]:
labelCat= ["gluon", "quark", "W", "Z", "top"]
# this function makes the histogram of a given quantity for the five classes
def makePlot_p(feature_index, input_data, input_featurenames):
    plt.subplots()
    for i in range(len(labelCat)):
        my_data = input_data[:,:,feature_index]
        # notice the use of numpy masking to select specific classes of jets
        my_data = my_data[np.argmax(target, axis=1) == i]
        # then plot the right quantity for the reduced array
        plt.hist(my_data[:,feature_index].flatten(), 50, density=True, histtype='step', fill=False, linewidth=1.5)
    plt.yscale('log')    
    plt.legend(labelCat, fontsize=12, frameon=False)  
    plt.xlabel(str(input_featurenames[feature_index], "utf-8"), fontsize=15)
    plt.ylabel('Prob. Density (a.u.)', fontsize=15)
    plt.show()
    #del fig, ax
    #return fig, ax

## Generative model of jet images with AE

In [11]:
import keras
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

# Importing keras and related modules
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Input,Dense,Flatten,Dropout,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
from tensorflow.keras.utils import to_categorical

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt

tf.config.experimental.set_memory_growth(tf.config.experimental.list_physical_devices('GPU')[0], True)
tf.keras.backend.clear_session()


# Data Preprocessing


In [13]:
# Load and preprocess jet image data (replace with your own data loading)
# Assuming you have jet images in a numpy array 'jet_images'
jet_images = np.array(f.get('jetImage')).astype('float32') / 255.0
num_images, image_size, _ = jet_images.shape


In [14]:
# Flatten the images and normalize pixel values
flattened_images = jet_images.reshape(num_images, -1)
flattened_images = (flattened_images - np.min(flattened_images)) / (np.max(flattened_images) - np.min(flattened_images))


In [15]:
# Build a simple 2D Normalizing Flow model with fewer parameters

# Define a simple transformation layer
class TransformationLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(TransformationLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        super(TransformationLayer, self).build(input_shape)
    
    def call(self, inputs):
        mean = Dense(1, activation='linear')(inputs)
        log_scale = Dense(1, activation='linear')(inputs)
        scale = tf.exp(log_scale)
        transformation = mean + scale * inputs
        log_det_jacobian = tf.reduce_sum(log_scale, axis=1)
        return transformation, log_det_jacobian


In [16]:
# Create a Normalizing Flow model with multiple transformation layers
input_layer = Input(shape=(flattened_images.shape[1],))
x = input_layer
num_layers = 3
for _ in range(num_layers):
    x, log_det_jacobian = TransformationLayer()(x)

normalizing_flow_model = Model(input_layer, x)

In [19]:
# Clear memory and resources
tf.keras.backend.clear_session()
# Explicitly set device
with tf.device('/GPU:0'):  # Change to '/CPU:0' if needed
    # Build the model
    input_shape = (flattened_images.shape[1],)
    input_layer = Input(shape=input_shape)
    x = Dense(32, activation='relu')(input_layer)
    output_layer = Dense(input_shape[0], activation='linear')(x)
    model = Model(input_layer, output_layer)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError())

In [21]:
# Train the model
model.fit(flattened_images, flattened_images, epochs=10, batch_size=64)


2023-08-16 02:02:26.570784: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 381.47MiB (rounded to 400000000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-08-16 02:02:26.570967: W tensorflow/core/common_runtime/bfc_allocator.cc:474] *******************************************************************************************xxxxx****


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
# Generate samples using the learned model
samples = np.random.randn(num_images, flattened_images.shape[1])
generated_samples = normalizing_flow_model.predict(samples)

# Reshape generated samples back to image shape
generated_images = generated_samples.reshape(num_images, image_size, image_size)


In [ ]:
# Visualize the generated images
for i in range(5):  # Display first 5 generated images
    plt.imshow(generated_images[i], cmap='gray')
    plt.title('Generated Image')
    plt.show()